![head.png](https://github.com/iwh-halle/FinancialDataAnalytics/blob/master/figures/head.jpg?raw=1)

# Financial Data Analytics in Python

**Prof. Dr. Fabian Woebbeking**</br>
Assistant Professor of Financial Economics

IWH - Leibniz Institute for Economic Research</br>
MLU - Martin Luther University Halle-Wittenberg

fabian.woebbeking@iwh-halle.de

# Homework

You will need a Git/GitHub repository to submit your course deliverables. Consult [**slides.ipynb**](https://github.com/iwh-halle/FinancialDataAnalytics) for help with the tasks below! If you need further assistance, do not hesitate to open a Q&A at https://github.com/cafawo/FinancialDataAnalytics/discussions

### Task: 

The liquidity position of a firm measured in million is a **generalized Wiener process** with a drift of $0.1$ per month and a variance of $\sigma^2 = 0.16$ per month. The initial cash position is $2.0$. Calculate:

1. 	the expected value and standard deviation in one, six and 12 months.
2.	What is the probability of a negative cash position in 6 and 12 months? 

In [1]:
import numpy as np
from scipy.stats import norm

# Given values
mu = 0.1  # drift per month
sigma = np.sqrt(0.16)  # standard deviation per month
X_0 = 2.0  # initial cash position

# Time periods in months
times = [1, 6, 12]

# Calculate expected values and standard deviations
for t in times:
    expected_value = X_0 + mu * t
    std_deviation = sigma * np.sqrt(t)
    print(f"At {t} months:")
    print(f"  Expected Value: {expected_value:.4f} million")
    print(f"  Standard Deviation: {std_deviation:.4f} million")
    
    if t == 6 or t == 12:
        # Calculate the probability of a negative cash position
        z_value = (0 - expected_value) / std_deviation
        prob_negative_cash = norm.cdf(z_value)
        print(f"  Probability of Negative Cash Position: {prob_negative_cash:.4%}")


At 1 months:
  Expected Value: 2.1000 million
  Standard Deviation: 0.4000 million
At 6 months:
  Expected Value: 2.6000 million
  Standard Deviation: 0.9798 million
  Probability of Negative Cash Position: 0.3982%
At 12 months:
  Expected Value: 3.2000 million
  Standard Deviation: 1.3856 million
  Probability of Negative Cash Position: 1.0461%


### Task: 

The cash flow of a [call option](https://en.wikipedia.org/wiki/Call_option) with strike $K$ at maturity $T$ is given by

$$
max(S_T - K, 0) = (S_T - K)^+
$$

where $S_T$ is the price of the underlying at $T$. The price of the option under the [risk-neutral measure](https://en.wikipedia.org/wiki/Risk-neutral_measure) $\mathbb{Q}$ is simply its discounted expected value
$$
\mathbb{E}^\mathbb{Q}[(S_T - K)^+] e^{-rT}.
$$


Calculate the price of the option, using:
1. numerical integration and
2. Monte carlo simulation.

For you calculations, assume that todays price of the underlying is $S_0 = 220$, the strike is $K = 220$, volatility is $\sigma = 0.98$, the risk free rate is $r = 10\%$ (continuous) and maturity is one year. We further assume that the underlying $S$ follows a **Geometric Brownian motion**.

Hint: The terminal stock price $S_T$, under the risk-neutral measure, follows a log-normal distribution with PDF

$$f(x) = \frac{1}{x s \sqrt{2 \pi}} \exp\left( -\frac{(\ln x - \mu)^2}{2 s^2} \right) $$

where $\mu = \ln S_0 + (r-\sigma^2 / 2)T$ and variance $s^2 = \sigma^2 T$.


In [2]:
import numpy as np
import scipy.integrate as integrate
import matplotlib.pyplot as plt

# Given values
S_0 = 220
K = 220
sigma = 0.98
r = 0.10
T = 1

# Parameters for the log-normal distribution
mu = np.log(S_0) + (r - 0.5 * sigma**2) * T
s = sigma * np.sqrt(T)

# Risk-free discount factor
discount_factor = np.exp(-r * T)

# 1. Numerical Integration
def integrand(ST):
    payoff = np.maximum(ST - K, 0)
    pdf = (1 / (ST * s * np.sqrt(2 * np.pi))) * np.exp(-((np.log(ST) - mu) ** 2) / (2 * s**2))
    return payoff * pdf

# Integrate over the range from 0 to a reasonably large number
integral_result, _ = integrate.quad(integrand, 0, S_0 * 10)
option_price_numerical_integration = integral_result * discount_factor
print(f"Option Price (Numerical Integration): {option_price_numerical_integration:.4f}")

# 2. Monte Carlo Simulation
np.random.seed(0)  # For reproducibility
num_simulations = 1000000
ST_simulations = S_0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * np.random.randn(num_simulations))
payoffs = np.maximum(ST_simulations - K, 0)
option_price_monte_carlo = np.mean(payoffs) * discount_factor
print(f"Option Price (Monte Carlo Simulation): {option_price_monte_carlo:.4f}")


Option Price (Numerical Integration): 81.5466
Option Price (Monte Carlo Simulation): 89.7523
